In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import math
import seaborn as sns
import json

In [2]:
%matplotlib inline

In [3]:
transaction_to_uprn_lookup = pd.read_csv('../data/ppdid_uprn_usrn.csv')
transaction_to_uprn_lookup.head()

,uprn,transactionid,parentuprn,usrn
0,30,{DD204FA6-B333-4A06-9E31-7ACABE162EB7},NaN,4549897
1,31,{B26F0F8D-4C7A-4DBF-B10A-3336830BF462},NaN,4549897
2,63,{241A4A53-DD8C-706B-E050-A8C06205638B},NaN,4582546
3,97,{85866A64-627D-143F-E053-6B04A8C06A15},NaN,4527985
4,103,{BE61D947-ED06-482E-8274-8B84912C15CB},NaN,4527985


In [4]:
transaction_to_uprn_lookup.shape

(25950983, 4)

In [5]:
import math
output_file = '../data/pp-complete-with-uprn.csv'
chunksize=100000
columns = ['id', 'price', 'date', 'postcode', 'property_type', 'old_new', 'duration', 'paon', 'saon', 'street', 'locality', 'town', 'district', 'county', 'ppd_category', 'record_status']
i = 0
with open(output_file, 'w') as f:
    f.write(','.join(columns + ['uprn', 'parentuprn', 'usrn']) + '\n')
for chunk in pd.read_csv('../data/pp-complete.csv', chunksize=chunksize, header=0, names=columns):
        df = chunk
        df['date'] = pd.to_datetime(df.date).apply(lambda x: x.date())
        merged = df.merge(transaction_to_uprn_lookup, how='inner', left_on='id', right_on='transactionid').drop(columns='transactionid')
        merged.to_csv(output_file, mode="a", header=False, index=False)
        if i % 20 == 0:
            print("Done chunk {0}".format(i))
        i += 1

Done chunk 0
Done chunk 20
Done chunk 40
Done chunk 60
Done chunk 80
Done chunk 100
Done chunk 120
Done chunk 140
Done chunk 160
Done chunk 180
Done chunk 200
Done chunk 220
Done chunk 240
Done chunk 260
Done chunk 280


In [6]:
to_filter_with = pd.read_csv('../data/uprns_of_interest.csv', index_col=None)
to_filter_with.head()

/var/folders/zs/1hvc18dn7v94z77k1xy7g90c0000gn/T/ipykernel_47112/1284509313.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  to_filter_with = pd.read_csv('../data/uprns_of_interest.csv', index_col=None)


,"CHRISTCHURCH AND POOLE"""
0,1000
1,100000000027
2,100000000028
3,100000000031
4,100000000032


In [7]:
to_filter_with.columns = ['uprn']

In [12]:
import math
output_file = '../data/filtered_osopenuprn.csv'
chunksize=100000
input_columns = ['uprn', 'x_coord', 'y_coord', 'lat', 'lon']
i = 0
with open(output_file, 'w') as f:
    f.write(','.join(input_columns) + '\n')
for chunk in pd.read_csv('../data/osopenuprn_202310.csv', chunksize=chunksize, header=0, names=input_columns):
        df = chunk
        merged = to_filter_with.merge(df, how='inner', on='uprn')
        merged.to_csv(output_file, mode="a", index=False, header=False)
        if i % 20 == 0:
            print("Done chunk {0}".format(i))
        i += 1

Done chunk 0
Done chunk 20
Done chunk 40
Done chunk 60
Done chunk 80
Done chunk 100
Done chunk 120
Done chunk 140
Done chunk 160
Done chunk 180
Done chunk 200
Done chunk 220
Done chunk 240
Done chunk 260
Done chunk 280
Done chunk 300
Done chunk 320
Done chunk 340
Done chunk 360
Done chunk 380
Done chunk 400


In [13]:
del transaction_to_uprn_lookup
del to_filter_with

In [14]:
kept_lookup = pd.read_csv('../data/filtered_osopenuprn.csv', index_col=None)
kept_lookup.head()

,uprn,x_coord,y_coord,lat,lon
0,1000,358574.0,179525.0,51.513120,-2.598358
1,100003,361077.0,173226.0,51.456664,-2.561593
2,100007,361333.0,173225.0,51.456673,-2.557909
3,10001,354146.0,178238.0,51.501206,-2.661996
4,100012,361250.0,173225.0,51.456667,-2.559103


In [19]:
merged.head()

,id,price,date,postcode,property_type,old_new,duration,paon,saon,street,...,county,ppd_category,record_status,uprn,parentuprn,usrn,x_coord,y_coord,lat,lon
0,{40FD4DF2-5362-407C-92BC-566E2CCE89E9},44500,1995-02-03,SR6 0AQ,T,N,F,50,NaN,HOWICK PARK,...,TYNE AND WEAR,A,A,45049391,NaN,38734115,439764.0,557681.0,54.912270,-1.381292
1,{7A99F89E-7D81-4E45-ABD5-566E49A045EA},56500,1995-01-13,CO6 1SQ,T,N,F,19,NaN,BRICK KILN CLOSE,...,ESSEX,A,A,100090310217,NaN,3900242,585448.0,223310.0,51.877526,0.692765
2,{28225260-E61C-4E57-8B56-566E5285B1C1},58000,1995-07-28,B90 4TG,T,N,F,37,NaN,RAINSBROOK DRIVE,...,WEST MIDLANDS,A,A,100071006635,NaN,35005306,413632.0,276827.0,52.389317,-1.801110
3,{AE76CAF1-F8CC-43F9-8F63-4F48A2857D41},17000,1995-03-10,S65 1QJ,T,N,L,22,NaN,DENMAN STREET,...,SOUTH YORKSHIRE,A,A,100050830486,NaN,32601135,443444.0,393520.0,53.436695,-1.347514
4,{709FB471-3690-4945-A9D6-4F48CE65AAB6},58000,1995-04-28,PE7 3AL,D,Y,F,4,NaN,BROOK LANE,...,CAMBRIDGESHIRE,A,A,10000154669,NaN,17804028,519984.0,294640.0,52.536432,-0.232422


In [21]:
import math
output_file = '../data/pp-complete-with-location.csv'
chunksize=100000
input_columns = ['uprn', 'x_coord', 'y_coord', 'lat', 'lon']
columns = ['id', 'price', 'date', 'postcode', 'property_type', 'old_new', 'duration', 'paon', 'saon', 'street', 'locality', 'town', 'district', 'county', 'ppd_category', 'record_status', 'uprn','parentuprn','usrn']
i = 0
with open(output_file, 'w') as f:
    f.write(','.join(columns + ['x_coord', 'y_coord', 'lat', 'lon']) + '\n')
for chunk in pd.read_csv('../data/pp-complete-with-uprn.csv', chunksize=chunksize, header=0, names=columns):
        df = chunk
        merged = df.merge(kept_lookup, how='inner', on='uprn')
        merged.to_csv(output_file, mode="a", index=False, header=False)
        if i % 20 == 0:
            print("Done chunk {0}".format(i))
        i += 1

Done chunk 0
Done chunk 20
Done chunk 40
Done chunk 60
Done chunk 80
Done chunk 100
Done chunk 120
Done chunk 140
Done chunk 160
Done chunk 180
Done chunk 200
Done chunk 220
Done chunk 240
